# Autoencoder

## Imports and functions

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD

from functools import partial
import re
import os
import time
import tensorboard

from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, Reshape
from tensorflow.keras.layers import Input,Flatten,Dense, Dropout, GlobalAveragePooling2D, AveragePooling1D
from tensorflow.keras import models
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
def load_files_list(directory):
    '''
    Load the audiofiles inside a directory.
    '''
    return os.listdir(directory)

In [3]:
def normalize1(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [4]:
def normalize2(v):
    return (v - np.min(v)) / (np.max(v)-np.min(v))

In [5]:
def normalize3(v, m, var):
    return (v - m) / var

## Getting the descriptors

In [6]:
DIRECTORY = '/home/belen_alastruey/PYTHON/Autoencoder/'

In [7]:
train_path = DIRECTORY + 'Descriptors/'

In [8]:
files = load_files_list(train_path)

### Read .npy

In [9]:
descriptors = []
for file in files:
    if "fan" in file:
        descr = np.load(train_path + file)
        descriptors.append(descr)

In [10]:
descr = np.asarray(descriptors)

In [12]:
normalized=[]
m = np.mean(descr)
var=np.std(descr)
for v in descriptors:
    normalized.append(normalize2(v))
    #normalized.append(normalize3(v, m, var))

In [13]:
descr = np.asarray(normalized)

### Train - Test split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test= train_test_split(descr,test_size=0.3, shuffle = True, random_state=100)

### Reshape for convolutional autoencoder

In [18]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[2],1)

## Builiding the autoencoder

### Encoder 1

In [19]:
encoder = Sequential([keras.Input(shape=( 1, 128))])

In [20]:
#Layers
encoder.add(Dense(128, activation = "tanh"))
encoder.add(Dropout(0.1))
encoder.add(Dense(64, activation = "tanh"))
encoder.add(Dropout(0.1))
encoder.add(Dense(32, activation = "tanh"))
encoder.add(Dropout(0.1))
encoder.add(Dense(16, activation = "tanh"))
encoder.add(Dropout(0.1))
encoder.add(Dense(8, activation = "tanh"))

### Encoder 2

In [19]:
encoder = Sequential([keras.Input(shape=( 128, 1))])

In [20]:
encoder.add(Conv1D(8,3,activation='tanh', padding = "same"))
encoder.add(MaxPooling1D(2))
encoder.add(Conv1D(4,3,activation='tanh', padding = "same"))
encoder.add(MaxPooling1D(2))
encoder.add(AveragePooling1D())
encoder.add(Flatten())
encoder.add(Dense(2, activation = "tanh"))

### Summary

In [21]:
encoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 128, 8)            32        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 64, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 4)             100       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 4)             0         
_________________________________________________________________
average_pooling1d (AveragePo (None, 16, 4)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
Total params: 132
Trainable params: 132
Non-trainable params: 0
__________________________________________________________

### Decoder 1

In [22]:
decoder = Sequential([keras.Input(shape=(1,8))])

In [33]:
decoder.add(Dense(16, activation = "tanh"))
decoder.add(Dropout(0.1))
decoder.add(Dense(32, activation = "tanh"))
decoder.add(Dropout(0.1))
decoder.add(Dense(64, activation = "tanh"))
decoder.add(Dropout(0.1))
decoder.add(Dense(128, activation = "tanh"))
decoder.add(Dense(128, activation = "sigmoid"))

### Decoder 2

In [22]:
decoder = Sequential([keras.Input(shape=(64))])

In [23]:
decoder.add(Dense(64, activation = "tanh"))
decoder.add(Reshape((16,4), input_shape = (64,)))
decoder.add(Conv1D(4,1, strides =1, activation='tanh', padding = "same"))
decoder.add(UpSampling1D(2))
decoder.add(Conv1D(8,1, strides =1, activation='tanh', padding = "same"))
decoder.add(UpSampling1D(2))
decoder.add(UpSampling1D(2))
decoder.add(Conv1D(1,1, strides =1, activation='sigmoid', padding = "same"))

### Summary

In [24]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 16, 4)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 4)             20        
_________________________________________________________________
up_sampling1d (UpSampling1D) (None, 32, 4)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 32, 8)             40        
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 64, 8)             0         
_________________________________________________________________
up_sampling1d_2 (UpSampling1 (None, 128, 8)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 128, 1)           

### Autoencoder

In [25]:
autoencoder = Sequential([keras.Input(shape=(128,1))])

In [26]:
autoencoder.add(encoder)
autoencoder.add(decoder)

In [27]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [28]:
autoencoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 64)                132       
_________________________________________________________________
sequential_1 (Sequential)    (None, 128, 1)            69        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


## Training

In [29]:
nb_epochs = 100
bs = 30

### Without tensorboard

In [24]:
cb = [EarlyStopping(monitor='accuracy', patience=8, verbose=1, mode='auto')]

In [ ]:
autoencoder.fit(X_train, X_train,
                epochs= nb_epochs,
                batch_size= bs)
#validation_data=(X_test, X_test)

### With tensorboard

In [30]:
import tensorboard
#import run_logdir from tensorboard

In [31]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    run_id = time.strftime("autoencoder_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'

In [32]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [33]:
#Tensorboard, extra parameter: tensorboard --logdir="my_logs/"
autoencoder.fit(X_train, X_train, epochs=nb_epochs, batch_size=bs, validation_data=(X_test, X_test),callbacks=[tensorboard_cb])

Epoch 1/100
 1/72 [..............................] - ETA: 0s - loss: 0.1959 - mae: 0.4259WARNING:tensorflow:From /home/belen_alastruey/PYTHON/envs/pythonIntro/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
72/72 [==============================] - 1s 8ms/step - loss: 0.1229 - mae: 0.3279 - val_loss: 0.0605 - val_mae: 0.2214
Epoch 2/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0464 - mae: 0.1754 - val_loss: 0.0412 - val_mae: 0.1508
Epoch 3/100
72/72 [==============================] - 0s 6ms/step - loss: 0.0404 - mae: 0.1432 - val_loss: 0.0400 - val_mae: 0.1381
Epoch 4/100
72/72 [==============================] - 0s 6ms/step - loss: 0.0399 - mae: 0.1358 - val_loss: 0.0398 - val_mae: 0.1343
Epoch 5/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0397 - mae:

Epoch 59/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1231 - val_loss: 0.0343 - val_mae: 0.1231
Epoch 60/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1231 - val_loss: 0.0343 - val_mae: 0.1228
Epoch 61/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1231 - val_loss: 0.0343 - val_mae: 0.1230
Epoch 62/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1231 - val_loss: 0.0343 - val_mae: 0.1225
Epoch 63/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1230 - val_loss: 0.0343 - val_mae: 0.1224
Epoch 64/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1230 - val_loss: 0.0343 - val_mae: 0.1232
Epoch 65/100
72/72 [==============================] - 0s 5ms/step - loss: 0.0343 - mae: 0.1230 - val_loss: 0.0343 - val_mae: 0.1224
Epoch 66/100
72/72 [==============================] - 0s 5ms/step - loss: 0.

## Evaluation 

In [34]:
score = autoencoder.evaluate(X_test, X_test)
print(score)

29/29 [==============================] - 0s 2ms/step - loss: 0.0342 - mae: 0.1224
[0.03420661762356758, 0.12235258519649506]


## Saving the model

In [35]:
autoencoder.save('convolucional3_eam0122.h5')